In [3]:
# Scrape note data from random website

import requests
import re
from bs4 import BeautifulSoup, PageElement

WEBSITE_NAME = 'https://pages.mtu.edu/~suits/notefreqs.html'

def convert_note_name(element: PageElement):
    text = re.sub(r'\s', '', element.text)
    
    # Attempt to match single note (eg. C4)
    result = re.match(r'^[CDEFGAB][0-8]$', text)
    if result:
        return text
    
    # Attempt to match sharp note
    result = re.match(r'^([CDEFGAB])#([0-8])', text)
    if result:
        return '{}{}_#'.format(result.group(1), result.group(2))
    
    raise ValueError('invalid note name')

def convert_note_frequency(element: PageElement):
    return float(element.text)

def scrape_page(page: BeautifulSoup):
    notes = []
    for row in soup.select_one('center > center > table ').select('tr'):
        children = list(filter(lambda c: c != '\n', row.contents))
        notes.append({
            'name': convert_note_name(children[0]),
            'frequency': convert_note_frequency(children[1])
        })

    return notes

response = requests.get(WEBSITE_NAME)
soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
notes = scrape_page(soup)

print('Successfully scraped {} notes from {}'.format(len(notes), WEBSITE_NAME))

Successfully scraped 108 notes from https://pages.mtu.edu/~suits/notefreqs.html


In [4]:
# Dump scraped data to .csv for backup

OUT_FILENAME = 'notes.csv'

with open(OUT_FILENAME, 'w') as outfile:
    outfile.write('Name,Frequency\n')
    for note in notes:
        outfile.write('{},{}\n'.format(note['name'], note['frequency']))

print('Successfully dumped {} notes to \"{}\"'.format(len(notes), OUT_FILENAME))

Successfully dumped 108 notes to "notes.csv"


In [13]:
# Calculate register values for notes
# The value calculated will be how many full sample buffers are outputted per second

SAMPLE_CLOCK_FREQUENCY = 30400000
SAMPLE_BUFFER_SIZE = 256

# Calculate frequency for a given divider
def div_to_freq(div: int):
    if not validate_div(div):
        raise ValueError('div must be an 11-bit unsigned integer')
    
    return SAMPLE_CLOCK_FREQUENCY / (SAMPLE_BUFFER_SIZE * (div + 1))

def validate_div(div: int):
    return not (div < 0 or div >= 2048)

# Calculate divider for a given frequency
def freq_to_div(freq: float):
    if freq <= 0:
        raise ValueError('freq must be a non-zero positive number')
    
    max_freq = div_to_freq(0)
    if freq > max_freq:
        raise ValueError('freq must be smaller than maximum frequency ({})'.format(max_freq))
    
    return (SAMPLE_CLOCK_FREQUENCY / (SAMPLE_BUFFER_SIZE * freq)) - 1

def find(pred, collection):
    return next(i for i in collection if pred(i))

divisors = []
successful_divs = 0
for note in notes:
    div = freq_to_div(note['frequency'])
    rounded_div = round(div)

    if not validate_div(rounded_div):
        divisors.append({
            'note_name': note['name'],
            'desired_frequency': note['frequency'],
            'divisor': rounded_div
        })
    else:
        successful_divs += 1
        rounded_freq = div_to_freq(rounded_div)
        divisors.append({
            'note_name': note['name'],
            'desired_frequency': note['frequency'],
            'actual_frequency': rounded_freq,
            'divisor': rounded_div,
            'frequency_difference': round(rounded_freq - note['frequency'], 2)
        })

difference_sorted = divisors.copy()
difference_sorted.sort(key=lambda d: abs(d['frequency_difference']) if 'frequency_difference' in d else 0)
difference_min = find(lambda d: 'frequency_difference' in d, difference_sorted)
difference_max = difference_sorted[-1]

print('Successfully calculated divisors for {} out of {} notes'.format(successful_divs, len(notes)))
print('    - Smallest frequency difference: {} Hz with {}'.format(round(difference_min['frequency_difference'], 2), difference_min['note_name']))
print('    - Largest frequency difference: {} Hz with {}'.format(round(difference_max['frequency_difference'], 2), difference_max['note_name']))

Successfully calculated divisors for 86 out of 108 notes
    - Smallest frequency difference: -0.0 Hz with A1_#
    - Largest frequency difference: -111.0 Hz with E8


In [17]:
# Dump divisor data into a .csv for easy viewing of data

OUT_FILENAME = 'divisors.csv'

with open(OUT_FILENAME, 'w') as outfile:
    outfile.write('Name,Divisor,Desired frequency,Actual frequency,Frequency difference\n')
    for divisor in divisors:
        outfile.write('{},{},{},{},{}\n'.format(
            divisor['note_name'],
            divisor['divisor'],
            divisor['desired_frequency'],
            round(divisor['actual_frequency'], 2) if 'actual_frequency' in divisor else '',
            divisor['frequency_difference'] if 'frequency_difference' in divisor else ''
        ))
        
print('Successfully dumped {} divisors to \"{}\"'.format(len(divisors), OUT_FILENAME))

Successfully dumped 108 divisors to "divisors.csv"


In [24]:
# Dump register values to .csv for easy viewing of data

OUT_FILENAME = 'registers.csv'

def calculate_registers(div: int):
    lower = '0x%02X' % (div & 0xFF)
    upper = '0x-%X' % ((div >> 8) & 0x07)

    return lower, upper

with open(OUT_FILENAME, 'w') as outfile:
    outfile.write('Name,Divisor,0x00,0x01\n')
    for divisor in divisors:
        valid_note = 'actual_frequency' in divisor
        lower, upper = calculate_registers(divisor['divisor'])
        outfile.write('{},{},{},{}\n'.format(
            divisor['note_name'],
            divisor['divisor'],
            lower if valid_note else '',
            upper if valid_note else ''
        ))

print('Successfully dumped {} register values to \"{}\"'.format(len(divisors), OUT_FILENAME))

Successfully dumped 108 register values to "registers.csv"


In [34]:
# Dump divisor values into a .z80 assembly file to be linked with gameboy source

OUT_FILENAME = 'notes.z80'
FIRST_NOTE = 'C2'
TABLE_LABEL = 'note_table'

def index_pred(pred, collection):
    for i, e in enumerate(collection):
        if pred(e):
            return i
    return -1

valid_divisors = divisors[index_pred(lambda d: d['note_name'] == FIRST_NOTE, divisors):]

with open(OUT_FILENAME, 'w') as outfile:
    outfile.write('; {}\n;\n; Note table\n;\n'.format(OUT_FILENAME))
    outfile.write('; Info:\n;    Note count     : {}\n;    Byte size      : {}\n;    Starting note  : {}\n;    End note       : {}\n;\n'.format(
        len(valid_divisors),
        len(valid_divisors) * 2,
        FIRST_NOTE,
        valid_divisors[-1]['note_name']
    ))
    outfile.write('; generated by note-table.ipynb\n\n')
    outfile.write('section \"Note table\", rom0\n\n')
    outfile.write('; Start of the note table\n{}::\n'.format(TABLE_LABEL))

    for divisor in valid_divisors:
        outfile.write('dw $%04X    ; %s\n' % (divisor['divisor'] & 0x07FF, divisor['note_name']))

    outfile.write('\n; End of {}\n'.format(OUT_FILENAME))

print('Successfully dumped {} note values to \"{}\"'.format(len(valid_divisors), OUT_FILENAME))

Successfully dumped 84 note values to "notes.z80"
